## Setup spark 

In [1]:
import findspark
findspark.init('/afs/cern.ch/work/m/migliori/public/spark2.3.1')

In [2]:
application_name = 'dist-keras-notebook'
master = "local[*]"
num_executors = 5
num_cores = 4

## Map worker -> executor
num_workers = num_executors
print("Total number of workers: %d" % (num_workers) )

Total number of workers: 5


In [3]:
from pyspark.sql import SparkSession
import os 

os.environ["PYSPARK_SUBMIT_ARGS"] = "--packages org.diana-hep:spark-root_2.11:0.1.14 pyspark-shell"

spark = SparkSession.builder\
        .appName("test-spark-root")\
        .config("spark.driver.memory", "40G")\
        .getOrCreate()

To run this code on a cluster just change the SparkSession builder and the master

```Python
spark = SparkSession.builder\
        .appName(application_name)\
        .config("spark.pyspark.python", "/afs/cern.ch/work/m/migliori/public/anaconda2/bin/python")\
        .config("spark.master", master)\
        .config("spark.executor.cores", '{}'.format(num_cores))\
        .config("spark.executor.instances", '{}'.format(num_executors))\
        .getOrCreate()
```

In [4]:
spark

## Read and convert the samples

Create the vectors containing Low Level and High Level Features

In [5]:
from pyspark.sql.functions import lit
import time

from Utils_functions import *

PATH = 'data/small_sample/'
samples = ['qcd', 'ttbar', 'wjets']
labels = [0, 1, 2]

requiredColumns = ["EFlowTrack", "MuonTight_size", "Electron_size",
                   "EFlowNeutralHadron", "EFlowPhoton", "Electron",
                   "MuonTight", "MissingET", "Jet"]

data = None

start = time.time()
for sample, label in zip(samples, labels):
    print('Loading {} ...'.format(sample))
    
    #Load data in a temporary dataframe 
    df_tmp = spark.read \
        .format("org.dianahep.sparkroot.experimental") \
        .load(PATH+sample+'*.root') \
        .select(requiredColumns).toDF(*requiredColumns)
    
    #Count how many events there are
    print("There are {} {} events".format(df_tmp.count(), sample))
    
    ## Convert the dataframe and add the label
    print('Converting the dataframe ...')
    df_tmp_featured = df_tmp.rdd\
                    .map(convert)\
                    .filter(lambda row: len(row) > 0)\
                    .toDF() \
                    .withColumn("label", lit(label))
    
    # Merge the dataframe with the previous ones 
    if data==None:
        data = df_tmp_featured
    else:
        data = data.union(df_tmp_featured)
     
    # delete the tmp dataframe
    del df_tmp 

stop = time.time()
print('\nLoading and Converting the samples took {} s'.format(int(stop-start)))

Loading qcd ...
There are 47952 qcd events
Converting the dataframe ...
Loading ttbar ...
There are 479952 ttbar events
Converting the dataframe ...
Loading wjets ...
There are 480000 wjets events
Converting the dataframe ...

Loading and Converting the samples took 19 s


In [6]:
data.printSchema()

root
 |-- hfeatures: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- lfeatures: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: double (containsNull = true)
 |-- label: integer (nullable = false)



In [ ]:
## get the HLF dataset
HLF_dataset = data.select(['hfeatures', 'label'])
HLF_dataset.show(5)

+--------------------+-----+
|           hfeatures|label|
+--------------------+-----+
|[0.0, 1.844219446...|    0|
|[0.0, 28.26162528...|    0|
|[41.8011283874511...|    0|
|[84.4664764404296...|    0|
|[0.0, 16.53163719...|    0|
+--------------------+-----+
only showing top 5 rows



In [ ]:
print('There are {} events'.format(HLF_dataset.count()))

## Machine Learning 

**TO DO:**
* Split the dataset into training and validation (80/20%)
* One-Hot-Encode the label (dim=3)
* Scale the features to take values between 0 and 1 
    * Then apply the scaler also to the validation set
* Buld the NN:
    * 3 hidden layers with 50, 20 and 10 nodes activated by *ReLU*
    * Output layer with 3 nodes and *Softmax* activation
    * Use *categorical crossentropy* as a loss
    * Ask Maurizio for the optimizer, weight initialization, regularization, dropout
    * For now we can use *Adam* and leave everything else as default
* Create the trainer
    * AEASGD for now is the one with the best performances
* Train the model!

**Optional**
* ... Example of Cross Validation using spark?